Read in packages

In [1]:
import pandas as pd
import os
import numpy as np
import xarray as xr
import rioxarray
import glob
import random
import geopandas as gpd
from sklearn.utils import shuffle
from MightyMosaic import MightyMosaic
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold


In [8]:
df = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_poly_check.csv')

df['Keep'].value_counts(normalize=True) * 100

Keep
Yes      40.410315
No       30.324702
Maybe    29.264982
Name: proportion, dtype: float64

In [2]:
# Load your data
df = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_poly_check_ee.csv')
df = df[df['Keep'] == 'Yes']
df = df.rename(columns={'Image': 'ID'})
df = df[df['ID'].isin(['29690000000000-0000000000', '29690000000000-0000023296']) == False]
df['ID'] = df['ID'].astype(int)

# # Load shapefile data and merge with main dataframe
in_shape = gpd.read_file('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_polygons.shp')
in_shape['geometry'] = in_shape['geometry'].apply(lambda geom: geom.wkt)
in_shape = pd.DataFrame(in_shape[['Year', 'ID', 'area']])
df = pd.merge(in_shape, df, on='ID', how='inner')
df['Year'] = df['Year'].astype(int)
df = df[df['Keep'] == 'Yes']

df = df.reset_index(drop = True)

df.shape

/tmp/ipykernel_1984151/399835482.py:10: UserWarning: Geometry column does not contain geometry.
  in_shape['geometry'] = in_shape['geometry'].apply(lambda geom: geom.wkt)


(7740, 4)

In [9]:
import pandas as pd
import geopandas as gpd

# Load your data
df = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_poly_check_ee.csv')
df = df[df['Keep'] == 'Yes']
df = df.rename(columns={'Image': 'ID'})
df = df[~df['ID'].isin(['29690000000000-0000000000', '29690000000000-0000023296'])]
df['ID'] = df['ID'].astype(int)

# Load shapefile data and merge with main dataframe
in_shape = gpd.read_file('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_polygons.shp')
in_shape = in_shape[['Year', 'ID', 'geometry', 'area']]  # Ensure geometry column is included


in_shape = in_shape[in_shape['ID'] .isin (df['ID'])]


in_shape.to_file('/explore/nobackup/people/spotter5/cnn_mapping/Russia/good_polys_anna.shp')


In [8]:
# Load your data
df = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_poly_check_ee.csv')
df = df.rename(columns={'Image': 'ID'})
df = df[df['ID'].isin(['29690000000000-0000000000', '29690000000000-0000023296']) == False]
df['ID'] = df['ID'].astype(int)

# Load shapefile data and merge with main dataframe
in_shape = gpd.read_file('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_polygons.shp')
in_shape['geometry'] = in_shape['geometry'].apply(lambda geom: geom.wkt)
in_shape = pd.DataFrame(in_shape[['Year', 'ID', 'area']])
df = pd.merge(in_shape, df, on='ID', how='inner')
df['Year'] = df['Year'].astype(int)
df = df[df['Keep'] == 'Yes']

df = df.reset_index(drop = True)


df.head()
# Initialize KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Preparing indices for entire data shuffling
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split data into 5 folds
fold = 0
results = []
for train_val_index, test_index in kf.split(df_shuffled):
    # Extract test set (20% of total)
    test_df = df_shuffled.iloc[test_index]

    # Extract the remaining data for this fold
    train_val_df = df_shuffled.iloc[train_val_index]

    # Further split remaining data into training (70% of total, 87.5% of remaining) and validation (10% of total, 12.5% of remaining)
    train_df, val_df = train_test_split(train_val_df, test_size=0.125, random_state=42)  # 0.125 * 0.8 = 0.1 ~ 10% of total

    # Save each fold's datasets to CSV files
    train_df.to_csv(f'/explore/nobackup/people/spotter5/cnn_mapping/Russia/train_fold_{fold}.csv', index=False)
    val_df.to_csv(f'/explore/nobackup/people/spotter5/cnn_mapping/Russia/val_fold_{fold}.csv', index=False)
    test_df.to_csv(f'/explore/nobackup/people/spotter5/cnn_mapping/Russia/test_fold_{fold}.csv', index=False)

    fold += 1
    print(f"Fold {fold}: Train {train_df.shape}, Validation {val_df.shape}, Test {test_df.shape}")
    results.append((train_df.shape, val_df.shape, test_df.shape))


/tmp/ipykernel_3746922/2579618206.py:9: UserWarning: Geometry column does not contain geometry.
  in_shape['geometry'] = in_shape['geometry'].apply(lambda geom: geom.wkt)


Fold 1: Train (5418, 4), Validation (774, 4), Test (1548, 4)
Fold 2: Train (5418, 4), Validation (774, 4), Test (1548, 4)
Fold 3: Train (5418, 4), Validation (774, 4), Test (1548, 4)
Fold 4: Train (5418, 4), Validation (774, 4), Test (1548, 4)
Fold 5: Train (5418, 4), Validation (774, 4), Test (1548, 4)


In [20]:
test_id = test_df['ID']
train_id = train_df['ID']
val_id = val_df['ID']

#
common_elements = set(train_id).intersection(val_id)

# Convert the set back to a list, if needed
common_elements_list = list(common_elements)

print(common_elements_list)

[]


In [4]:
df.to_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/delete.csv')

In [5]:
df.shape

(7740, 4)

For some reason I don't have all the fires in my val sets, need to see why



In [14]:
import pandas as pd

one = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/test_fold_0.csv',  usecols=['ID'])
two = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/test_fold_1.csv',  usecols=['ID'])
three = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/test_fold_2.csv',  usecols=['ID'])
four = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/test_fold_3.csv',  usecols=['ID'])
five = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/test_fold_4.csv',  usecols=['ID'])





In [28]:
# Merge the dataframes on the 'ID' column
# merged = pd.merge(one, two, on='ID')
# merged = pd.merge(merged, three, on='ID')
# merged = pd.merge(merged, four, on='ID')
# merged = pd.merge(merged, five, on='ID')

merged = pd.concat([one, two, three, four, five])

# Get the common IDs
common_ids = merged['ID'].unique()
print(common_ids)

[10406 14758 16722 ...  7480  8800 13057]


In [29]:

#12450 ,12375. #12450 is in one
if 12450 in common_ids:
    
    print("in list")
else:
    print('not in list')

not in list


In [4]:
import geopandas as gpd

df = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_poly_check_ee.csv')
df = df.rename(columns={'Image': 'ID'})
df = df[df['ID'].isin(['29690000000000-0000000000', '29690000000000-0000023296']) == False]
df['ID'] = df['ID'].astype(int)

# Load shapefile data and merge with main dataframe
in_shape = gpd.read_file('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_polygons.shp')
in_shape['geometry'] = in_shape['geometry'].apply(lambda geom: geom.wkt)
in_shape = pd.DataFrame(in_shape[['Year', 'ID', 'area']])
df = pd.merge(in_shape, df, on='ID', how='inner')
df['Year'] = df['Year'].astype(int)
df = df[df['Keep'] == 'Yes']

df = df.reset_index(drop = True)
df.shape

/tmp/ipykernel_1932323/1530560583.py:10: UserWarning: Geometry column does not contain geometry.
  in_shape['geometry'] = in_shape['geometry'].apply(lambda geom: geom.wkt)


(7740, 4)

make sure all the merged ids are in df

In [15]:
in_test = merged['ID'].unique()

in_anna = df['ID'].unique()

result = [item for item in in_test if item not in in_anna]

result

[]

In [16]:
result2 = [item for item in in_anna if item not in in_test]
result2


[]

In [9]:
merged.head()

,ID
